## Noisy results

In [ ]:
%matplotlib widget

In [ ]:
# Import libraries
#import empymod
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import mu_0
import pandas as pd
import pygimli as pg

import sys
sys.path.insert(1, '../../src')

from PlotModel import PlotModelCond_3lay, PlotErrorSpace, PlotData, rmse_a
from EM1D import EMforward3lay

In [ ]:
# Import the conductivities and thicknesses used to create the LU table

conds = np.load('../data/conds.npy')
thick = np.load('../data/thicks.npy')

model_true = np.load('../data/B1_w/model_synth_w.npy')
data_true =  np.load('../data/B1_w/data_synth_w.npy')

# Load with noise
model_est_FS_noise2 = np.load('../data/B1_noise2_w/model_est_FS_B1_noise2_w.npy')
model_est_FS_noise5 = np.load('../data/B1_noise5_w/model_est_FS_B1_noise5_w.npy')
model_est_FS_noise10 = np.load('../data/B1_noise10_w/model_est_FS_B1_noise10_w.npy')

data_true_noise2 = np.load('../data/B1_noise2_w/data_synth_noise2_w.npy')
data_true_noise5 = np.load('../data/B1_noise5_w/data_synth_noise5_w.npy')
data_true_noise10 =  np.load('../data/B1_noise10_w/data_synth_noise10_w.npy')

npos = len(model_true)
nlayer = 3

depths_true = np.zeros((npos, nlayer))
sigmas_true = np.zeros((npos, nlayer))

depths_true[:,1] = model_true[:,3]
depths_true[:,2] = model_true[:,3] + model_true[:,4]
sigmas_true[:,:] = model_true[:,:3]

In [ ]:
fig, ax = plt.subplots(3,2)
PlotData(data_true, ax=ax)
PlotData(data_true_noise2, ax=ax)

In [ ]:
# Create forward modeller class

class EMforward_3lay_noise2(pg.Modelling):
    def __init__(self):
        """Initialize the model."""
        super().__init__()        
    def response(self, x):
        sig1 = x[0]
        sig2 = x[1]
        sig3 = x[2]
        thk1 = x[3]
        thk2 = x[4]
        if (thk1+thk2) >= 10:
            thk1 = 2
            thk2 = 3
        Z = EMforward3lay(sig1, sig2, sig3, thk1, thk2, height=0.47)                           
        return Z[:9]
    def createStartModel(self, dataVals):
        thk_ini = [2,3]
        sig_ini = [1, 1, 1]
        x0 = sig_ini + thk_ini
        return np.array(x0)

In [ ]:
# Initialize the forward modelling class
EMf_3lay_n2 = EMforward_3lay_noise2()

# Relative error array
error = 1e-4 # introduce here the error you want to test
relativeError = np.ones_like(data_true_noise2[0]) * error

# Define inversion transformations
transData = pg.trans.TransLog() # log transformation for data
transModel = pg.trans.TransLog()  # log-transform ensures thk and res >0

# Create inversion
invEM = pg.Inversion()
invEM.setForwardOperator(EMf_3lay_n2)

invEM.transModel = transModel
invEM.transData = transData

# Start inversion
model_est_noise2 = np.zeros_like(model_true)
#chi2hist = [] # empty array to store the chi2 history
#modelhist = [] # empty array to store the model history


In [ ]:
# Perform inversion for each 1D model per position in stitched section
for pos in range(npos):
    dataE = data_true_noise2[pos].copy()
    dataE *= np.random.randn(len(dataE)) * relativeError + 1.0
    model_est_pos = invEM.run(dataE, relativeError, verbose=False, lam=10, maxIter=40)
    #chi2hist.append(invEM.chi2History)
    #modelhist.append(invEM.modelHistory)
    model_est_noise2[pos] = model_est_pos